# Accident Analysis in Metro Nashville: 2013-2017 (Part 2)

### This notebook will:
* read in the cleaned data for accidents in Metro Nashville for 2013-2017,
* create data points for non-crash data,
* create a dataframe containing bootstrapped samples of crash/non-crash data,
* store that dataframe in a csv to be used for forecast model in Part 3.


#### Import packages

In [1]:
import glob
import os
import random
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline 
import datetime
import time as tm
import numpy as np
from time import gmtime, strftime, localtime

pd.options.mode.chained_assignment = None


#### Run script from Part 1 to get functions, variables into this notebook

In [2]:
%run ./'Nashville Accident Analysis -- Cleanup'.ipynb

CPU times: user 1.03 s, sys: 156 ms, total: 1.18 s
Wall time: 1.2 s


#### Read in data from CSV created in Part 1

In [3]:
non_empty_crash_data_df = pd.read_csv('data/Clean_Crash_Data.csv')
non_empty_crash_data_df.drop('Unnamed: 0',axis=1,inplace=True)
non_empty_crash_data_df

/Users/elancas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Accident Number        Date and Time  Number of Motor Vehicles  \
0           20130000050  2013-01-01 00:15:00                       2.0   
1           20130000270  2013-01-01 00:30:00                       2.0   
2           20130000128  2013-01-01 00:43:00                       2.0   
3           20130000123  2013-01-01 00:45:00                       2.0   
4           20130000160  2013-01-01 00:45:00                       2.0   
5           20130000142  2013-01-01 01:00:00                       2.0   
6           20130000138  2013-01-01 01:05:00                       1.0   
7           20130000206  2013-01-01 01:25:00                       2.0   
8           20130000306  2013-01-01 01:25:00                       2.0   
9           20130000393  2013-01-01 01:26:00                       1.0   
10          20130000249  2013-01-01 01:40:00                       2.0   
11          20130000240  2013-01-01 01:46:00                       2.0   
12          20130000255  2013-01-01 01:57:00                       1.0   
13          20130000315  2013-01-01 02:31:00                       1.0   
14          20130000379  2013-01-01 02:48:00                       2.0   
15          20130000391  2013-01-01 03:23:00                       1.0   
16          20130000403  2013-01-01 03:24:00                       2.0   
17          20130000408  2013-01-01 03:30:00                       2.0   
18          20130000414  2013-01-01 03:31:00                       2.0   
19          20130000407  2013-01-01 03:33:00                       1.0   
20          20130000532  2013-01-01 05:18:00                       1.0   
21          20130000564  2013-01-01 05:44:00                       1.0   
22          20130000637  2013-01-01 06:00:00                       1.0   
23          20130000662  2013-01-01 06:00:00                       1.0   
24          20130000626  2013-01-01 06:30:00                       1.0   
25          20130000627  2013-01-01 06:35:00                       1.0   
26          20130000635  2013-01-01 06:58:00                       1.0   
27          20130000651  2013-01-01 07:20:00                       1.0   
28          20130000671  2013-01-01 07:30:00                       2.0   
29          20130003167  2013-01-01 08:00:00                       2.0   
...                 ...                  ...                       ...   
147708      20171139178  2017-12-31 14:06:00                       2.0   
147709      20171139249  2017-12-31 14:52:00                       2.0   
147710      20171139257  2017-12-31 14:44:00                       2.0   
147711      20171139265  2017-12-31 14:59:00                       2.0   
147712      20171139374  2017-12-31 14:43:00                       2.0   
147713      20171139391  2017-12-31 16:04:00                       2.0   
147714      20171139424  2017-12-31 16:20:00                       2.0   
147715      20171139461  2017-12-31 16:30:00                       2.0   
147716      20171139474  2017-12-31 16:25:00                       2.0   
147717      20171139500  2017-12-31 17:00:00                       2.0   
147718      20171139543  2017-12-31 17:15:00                       2.0   
147719      20171139573  2017-12-31 17:40:00                       2.0   
147720      20171139578  2017-12-31 00:00:00                       2.0   
147721      20171139596  2017-12-31 18:04:00                       2.0   
147722      20171139628  2017-12-31 18:29:00                       2.0   
147723      20171139672  2017-12-31 18:49:00                       2.0   
147724      20171139674  2017-12-31 00:00:00                       2.0   
147725      20171139680  2017-12-31 18:31:00                       2.0   
147726      20171139683  2017-12-31 19:02:00                       2.0   
147727      20171139706  2017-12-31 19:10:00                       2.0   
147728      20171139794  2017-12-31 20:14:00                       1.0   
147729      20171139809  2017-12-31 20:21:00                       1.0   
147730      

We only have crash data, so trying to run it through a model by itself will be difficult, since the model only knows what constitutes crashes. To fix this, we'll derive non-crash data. Assuming the data we have is the full population of crashes, then we can take random timestamps and locations, pair them together, then go to the crash dataframe and filter on that pair. Most of the time, there won't be a crash at a randomly chosen time and place, so we record all those in a "random non-crashes" dataframe. Sometimes we will get lucky and actually grab a pair for which there was an accident(s). If we find those, we record them in a new "random crashes" dataframe. Then we concatenate them. Because the likelihood of grabbing a time/location pairing that results in finding accidents is so rare, we will have to bootstrap sample many many times to get enough crashes to get a representative set of crashes.

#### Grab a random 100,000 locations and timestamps, with replacement.

In [4]:
sample_size = 100000

In [5]:
random_locs = random.choices(list(non_empty_crash_data_df['Location']), k=sample_size)

In [6]:
random_times = random.choices(list(non_empty_crash_data_df['Rounded Date and Time']), k=sample_size)

#### For each location/timestamp pair, filter the crashes dataframe and if there are crashes, add them to a random crashes dataframe; otherwise, add a line to a random non-crashes dataframe that contains the time, lat/long, and best guess of street address for that location.

In [18]:
%%time

random_crashes_df = pd.DataFrame()
random_non_crashes_df = pd.DataFrame()

for idx in list(range(sample_size)):
    temp_df = non_empty_crash_data_df[(non_empty_crash_data_df['Rounded Date and Time'] == random_times[idx]) & 
                            (non_empty_crash_data_df['Location'] == random_locs[idx]) 
                           ]
    if temp_df.shape[0] > 0:
        random_crashes_df = random_crashes_df.append(temp_df.loc[:,['Rounded Date and Time',
                                                                    'Location',
                                                                    'Street Address',
                                                                    'Weather Description',
                                                                    'Precinct',
                                                                    'Zip',
                                                                    'City'
                                                                   ]])
    else:
        weather_description = non_empty_crash_data_df[(non_empty_crash_data_df['Rounded Date and Time'] == random_times[idx])]['Weather Description'].mode()[0]
        precinct = non_empty_crash_data_df[(non_empty_crash_data_df['Location'] == random_locs[idx])]['Precinct'].mode()[0]
        zip_code = non_empty_crash_data_df[(non_empty_crash_data_df['Location'] == random_locs[idx])]['Zip'].mode()[0]
        city = non_empty_crash_data_df[(non_empty_crash_data_df['Location'] == random_locs[idx])]['City'].mode()[0]
        street_address = non_empty_crash_data_df[(non_empty_crash_data_df['Location'] == random_locs[idx])]['Street Address'].mode()[0]
        random_non_crashes_df = random_non_crashes_df.append({'Rounded Date and Time': random_times[idx], 
                                                                          'Location': random_locs[idx],
                                                                          'Street Address': street_address,
                                                                          'Weather Description': weather_description,
                                                                            'Precinct': precinct,
                                                                            'Zip': zip_code,
                                                                            'City': city
                                                                          }, ignore_index=True)



CPU times: user 2h 14min 32s, sys: 58.2 s, total: 2h 15min 30s
Wall time: 2h 16min 49s


In [19]:
random_crashes_df

Rounded Date and Time                                  Location  \
87364    2016-03-18 16:00:00            (36.140190000000004, -86.7258)   
147115   2017-12-22 15:00:00                     (36.09483, -86.70694)   
22516    2013-12-04 04:00:00  (36.226315146602396, -86.60320828671992)   
122540   2017-04-07 17:00:00            (36.263059999999996, -86.7573)   
32545    2014-05-11 15:00:00                     (36.03888, -86.78281)   
60698    2015-05-18 07:00:00            (36.140190000000004, -86.7258)   
60715    2015-05-18 07:00:00            (36.140190000000004, -86.7258)   
140611   2017-10-18 17:00:00           (36.080090000000006, -86.72446)   
126223   2017-05-15 13:00:00                      (36.17904, -86.7737)   
100151   2016-08-08 00:00:00           (36.1693663201, -86.6798927725)   
54491    2015-03-04 17:00:00                     (36.19251, -86.82935)   
136916   2017-09-12 17:00:00            (36.14837, -86.77705999999999)   
20008    2013-10-29 13:00:00                     (36.30281, -86.69196)   
137862   2017-09-21 19:00:00            (36.11718, -86.72556999999999)   
119646   2017-03-06 20:00:00                     (36.15737, -86.78998)   
99958    2016-08-05 16:00:00                     (36.23775, -86.78721)   
94535    2016-06-06 13:00:00                     (36.02547, -86.58742)   
79521    2015-12-16 15:00:00                     (36.20643, -86.77637)   
44062    2014-10-22 12:00:00                     (36.09685, -86.65433)   
20770    2013-11-08 09:00:00                      (36.21891, -86.8464)   
136181   2017-09-03 13:00:00                       (36.1853, -86.6365)   
107798   2016-10-26 17:00:00             (36.290420000000005, -86.723)   
90715    2016-04-24 14:00:00                     (36.14342, -86.70313)   
69068    2015-08-26 17:00:00                      (36.2151, -86.59621)   
142900   2017-11-09 09:00:00  (36.085650688200005, -86.69982457620002)   
115901   2017-01-20 15:00:00           (36.045190000000005, -86.71543)   
105414   2016-10-01 20:00:00            (36.21815, -86.80348000000001)   
108049   2016-10-28 22:00:00            (36.10921, -86.72695999999999)   
141372   2017-10-26 06:00:00                      (36.1666, -86.76379)   
34676    2014-06-11 15:00:00    (36.0398572898454, -86.64920569887121)   
...                      ...                                       ...   
98234    2016-07-18 16:00:00           (36.014340000000004, -87.04043)   
102088   2016-08-28 00:00:00  (36.148540000000004, -86.66514000000001)   
123881   2017-04-21 16:00:00                     (36.15829, -86.77189)   
135519   2017-08-28 05:00:00            (36.06963, -86.69440999999999)   
101091   2016-08-18 14:00:00                     (36.23775, -86.78721)   
91544    2016-05-03 17:00:00       (36.0691722714, -86.68375906600001)   
50307    2015-01-12 07:00:00                      (36.26934, -86.7562)   
147589   2017-12-29 16:00:00                     (36.16757, -86.76555)   
79229    2015-12-14 08:00:00                      (36.16778, -86.6637)   
5362     2013-03-28 15:00:00    (36.1558104500686, -86.62940126227221)   
45686    2014-11-11 17:00:00           (36.050779999999996, -86.64692)   
2296     2013-02-08 06:00:00     (36.2362994853797, -86.7626967328179)   
84642    2016-02-16 16:00:00                     (36.16717, -86.82058)   
116022   2017-01-22 14:00:00                      (36.04899, -86.6651)   
97771    2016-07-13 07:00:00                     (36.18875, -86.79295)   
132750   2017-07-29 21:00:00                     (36.08416, -86.70589)   
20819    2013-11-08 18:00:00     (36.1703490013389, -86.6014201874848)   
62730    2015-06-11 16:00:00            (36.16905, -86.77226999999999)   
62734    2015-06-11 16:00:00            (36.16905, -86.77226999999999)   
113682   2016-12-28 08:00:00                     (36.10742, -86.71966)   
109516   2016-11-15 07:00:00                     (36.14376, -86.82261)   
132610   2017-07-28 14:00:00                      (36.11611, -86.7967)   
92320    20

#### For the two new dataframes, add a column for Crash Recorded and add a 1 for all crashes and a 0 for all non-crashes.

In [20]:
random_crashes_df['Crash Recorded'] = 1
random_non_crashes_df['Crash Recorded'] = 0

In [21]:
random_crashes_df.shape

(133, 8)

In [22]:
random_non_crashes_df.shape

(99872, 8)

#### Because there are so many more non-crashes than crashes, upsample the crashes to maintain class balance.

In [24]:
#this will upsample:
random_crashes_upsample_df = pd.DataFrame(random.choices(random_crashes_df.values, k=sample_size), columns=list(random_crashes_df.columns))
random_crashes_upsample_df

Rounded Date and Time                                  Location  \
0       2015-08-14 15:00:00                     (36.15477, -86.77942)   
1       2017-10-18 17:00:00           (36.080090000000006, -86.72446)   
2       2017-05-31 17:00:00                     (36.14439, -86.69475)   
3       2016-10-14 15:00:00                     (36.10742, -86.71966)   
4       2017-05-31 17:00:00                     (36.14439, -86.69475)   
5       2017-11-18 08:00:00                     (36.14895, -86.77942)   
6       2015-06-11 16:00:00            (36.16905, -86.77226999999999)   
7       2013-02-28 17:00:00                     (36.14723, -86.74364)   
8       2017-11-09 09:00:00  (36.085650688200005, -86.69982457620002)   
9       2015-12-17 17:00:00                     (36.15463, -86.77916)   
10      2015-12-17 17:00:00                     (36.15463, -86.77916)   
11      2016-05-17 16:00:00                     (36.14895, -86.77942)   
12      2016-10-20 16:00:00     (36.0818513597842, -86.7612235937558)   
13      2017-12-19 16:00:00                     (36.10742, -86.71966)   
14      2015-03-04 17:00:00                     (36.19251, -86.82935)   
15      2013-02-08 06:00:00     (36.2362994853797, -86.7626967328179)   
16      2016-03-18 16:00:00            (36.140190000000004, -86.7258)   
17      2016-04-25 10:00:00            (36.10621, -86.67305999999999)   
18      2015-07-02 07:00:00            (36.17222, -86.76888000000001)   
19      2015-11-24 09:00:00                      (36.1664, -86.76467)   
20      2013-12-04 04:00:00  (36.226315146602396, -86.60320828671992)   
21      2016-07-13 07:00:00                     (36.18875, -86.79295)   
22      2015-06-11 16:00:00            (36.16905, -86.77226999999999)   
23      2017-10-26 06:00:00                      (36.1666, -86.76379)   
24      2017-11-09 17:00:00                     (36.15737, -86.78998)   
25      2015-06-11 16:00:00            (36.16905, -86.77226999999999)   
26      2014-03-22 23:00:00                     (36.18388, -86.79821)   
27      2015-12-19 00:00:00            (36.04477, -86.66150999999999)   
28      2015-06-11 16:00:00            (36.16905, -86.77226999999999)   
29      2016-06-06 13:00:00                     (36.02547, -86.58742)   
...                     ...                                       ...   
99970   2017-08-28 05:00:00            (36.06963, -86.69440999999999)   
99971   2017-10-28 02:00:00                      (36.1189, -86.72644)   
99972   2015-06-11 16:00:00            (36.16905, -86.77226999999999)   
99973   2016-04-24 14:00:00                     (36.14342, -86.70313)   
99974   2017-06-23 18:00:00           (36.080859999999994, -86.72674)   
99975   2017-04-21 16:00:00                     (36.15829, -86.77189)   
99976   2017-10-16 10:00:00                     (36.15218, -86.76689)   
99977   2016-05-17 16:00:00                     (36.14895, -86.77942)   
99978   2015-04-02 16:00:00                     (36.09008, -86.70309)   
99979   2016-11-16 17:00:00            (36.17991, -86.80046999999999)   
99980   2017-11-09 09:00:00  (36.085650688200005, -86.69982457620002)   
99981   2014-06-11 14:00:00     (36.206296423887, -86.77663182277331)   
99982   2016-10-01 20:00:00            (36.21815, -86.80348000000001)   
99983   2013-05-24 09:00:00     (36.086045548889, -86.69994904166549)   
99984   2016-01-21 19:00:00                     (36.15385, -86.75708)   
99985   2016-12-14 07:00:00           (36.238279999999996, -86.75631)   
99986   2015-04-09 14:00:00           (36.144040000000004, -86.70125)   
99987   2014-12-05 09:00:00           (36.186040000000006, -86.69105)   
99988   2015-09-03 15:00:00            (36.15142, -86.75168000000001)   
99989   2015-04-02 16:00:00                     (36.09008, -86.70309)   
99990   2013-11-08 09:00:00                      (36.21891, -86.8464)   
99991   2016-08-01 20:00:00                     (36.06667, -86.71794)   
99992   2017-06-15 21:00:00            (36.10921, -86.7269599999

#### ONE-HOT ALL: Merge the two dataframes into one, and then extract features.

In [25]:
def final_data_prep(df1, df2):
    df_out = pd.concat([df1, df2])
    df_out['Rounded Date and Time'] = pd.to_datetime(df_out['Rounded Date and Time'])
    extract_time_features(df_out)
    extract_address_features(df_out)
    df_temp = df_out.loc[:,['Weather Description', 'Precinct', 'Zip', 'City', 'Day Of Week', 
                            #'Day Of Month',
                            'Month', 'Hour', 
                            #'Week'
                           ]].astype('category')
    df_temp_2 = df_out.loc[:,['Weekday', 'Daytime', 'Rush Hour Morning', 'Rush Hour Afternoon', 'Ramp',
            'Intersection', 'Interstate','Crash Recorded']]
    df_temp = pd.get_dummies(df_temp)
    df_out = pd.concat([df_temp, df_temp_2],axis=1)
    return df_out

In [26]:
model_data_df = final_data_prep(random_non_crashes_df,random_crashes_upsample_df)

/Users/elancas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


#### Write this out to a CSV that can be used later to train a model.

In [27]:
model_data_df.to_csv('data/Random_Model_Data_All_One_Hot.csv')

In [28]:
model_data_df

Weather Description_BLOWING SAND/SOIL/DIRT  \
0                                               0   
1                                               0   
2                                               0   
3                                               0   
4                                               0   
5                                               0   
6                                               0   
7                                               0   
8                                               0   
9                                               0   
10                                              0   
11                                              0   
12                                              0   
13                                              0   
14                                              0   
15                                              0   
16                                              0   
17                                              0   
18                                              0   
19                                              0   
20                                              0   
21                                              0   
22                                              0   
23                                              0   
24                                              0   
25                                              0   
26                                              0   
27                                              0   
28                                              0   
29                                              0   
...                                           ...   
99970                                           0   
99971                                           0   
99972                                           0   
99973                                           0   
99974                                           0   
99975                                           0   
99976                                           0   
99977                                           0   
99978                                           0   
99979                                           0   
99980                                           0   
99981                                           0   
99982                                           0   
99983                                           0   
99984                                           0   
99985                                           0   
99986                                           0   
99987                                           0   
99988                                           0   
99989                                           0   
99990                                           0   
99991                                           0   
99992                                           0   
99993                                           0   
99994                                           0   
99995                                           0   
99996                                           0   
99997                                           0   
99998                                           0   
99999                                           0   

       Weather Description_BLOWING SNOW  Weather Description_CLEAR  \
0                                     0                          1   
1                                     0                          0   
2                                     0                          0   
3                                     0                          0   
4                                     0                          1   
5                                     0                          1   
6                                     0                          1   
7                                     0                          1   
8                                     0                          1   
9                   